### CONFIGURATION

In [146]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [147]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [157]:
#VCF
VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/merckReruns/"

# BED
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
#BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
#BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = True
LOAD_NON_BED_MUTATIONS = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [158]:
def loadHealthChecksAndMerge(df):
    hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
    hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
    hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
    hc['SAMPLE']=hc['SAMPLE'].str[:12]
    hc = hc.set_index(['SAMPLE'])
    df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
    df.fillna("Unknown", inplace=True)
    return df

In [159]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    #if x[-10:] == "sliced.vcf":
    #    VCF_SAMPLES.append(x[40:52] + "R_" + x[40:52] + "T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf")
    if x[-10:] == "melted.vcf":
        VCF_SAMPLES.append(x)
print VCF_SAMPLES

#VCF_SAMPLES = ['CPCT02010384R_CPCT02010384T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf']

['CPCT02010090R_CPCT02010090T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010214R_CPCT02010214TIII_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010233R_CPCT02010233T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010246R_CPCT02010246T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010249R_CPCT02010249T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010263R_CPCT02010263T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010268R_CPCT02010268T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010269R_CPCT02010269T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010272R_CPCT02010272T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010275R_CPCT02010275T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010277R_CPCT02010277T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010298R_CPCT02010298T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf', 'CPCT02010299R_CPCT02010

<h3> LOAD VCF

In [160]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[14:len(VCF_SAMPLE)-50]:'melted'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVariantsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True,True,bed,LOAD_NON_BED_MUTATIONS)])
    else:
        df = pd.concat([df,aVCF.loadVariantsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[:12],True)])

reading BED file
Bed File Loaded
reading vcf file: CPCT02010090R_CPCT02010090T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 73078
reading BED file
Bed File Loaded
reading vcf file: CPCT02010214R_CPCT02010214TIII_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 67122
reading BED file
Bed File Loaded
reading vcf file: CPCT02010233R_CPCT02010233T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 52712
reading BED file
Bed File Loaded
reading vcf file: CPCT02010246R_CPCT02010246T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 52952
reading BED file
Bed File Loaded
reading vcf file: CPCT02010249R_CPCT02010249T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
Number variants loaded: 49168
reading BED file
Bed File Loaded
reading vcf file: CPCT02010263R_CPCT02010263T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf
reading VCF File line: 100000
reading VCF File line: 200000
Number varian

In [161]:
df = loadHealthChecksAndMerge(df)

In [162]:
dfPatients = pd.DataFrame([x[:12] for x in VCF_SAMPLES])
dfPatients.columns = (['patientName'])
dfPatients = pd.merge(dfPatients,hc,how='left', left_on='patientName',right_index=True)
dfPatients.fillna("Unknown", inplace=True)
dfPatients = pd.DataFrame(dfPatients.groupby(['TUMOR_TYPE']).agg('count')['patientName'])
dfPatients.rename(columns={'patientName':'patientCount'},inplace = True)

<h3> High Level Stats

In [164]:
dfPatients

,patientCount
TUMOR_TYPE,
Bladder cancer,1
Breast cancer,4
Gallbladder cancer,1
Gastrointestinal tract cancer,2
Kidney cancer,1
Liver cancer,1
Lung cancer: NSCLC,5
Lung cancer: Other,1
Prostate cancer,3


### HIGH and MODERATE impact NON-missense

In [ ]:
pd.pivot_table(df[((df['annWorstImpact']=='MODERATE')|(df['annWorstImpact']=='HIGH'))&(~df.annAllEffects.str.contains("missense"))], values='pos', index=['chromPos','patientName','TUMOR_TYPE','annGene','annWorstImpact','annWorstEffect','consensus','meltedAF'], columns=['inCOSMIC'], aggfunc='count',margins=True).fillna("")

### Missense

In [175]:
# Y CHROM
pd.pivot_table(df[(df.annAllEffects.str.contains("missense"))&(df.chrom == 'Y')], values='pos', index=['patientName','chromPos'], columns=['consensus','variantType'], aggfunc='count',margins=True).fillna("")

consensus               False   All
variantType               SNP      
patientName  chromPos              
CPCT02010090 Y:13500769   1.0   1.0
CPCT02010268 Y:4968557    1.0   1.0
CPCT02010277 Y:13506575   1.0   1.0
             Y:13506585   1.0   1.0
CPCT02010299 Y:13496413   1.0   1.0
             Y:13496428   1.0   1.0
CPCT02010306 Y:13500768   1.0   1.0
             Y:13500769   1.0   1.0
             Y:13500776   1.0   1.0
             Y:21903684   1.0   1.0
CPCT02010313 Y:13501688   1.0   1.0
All                      11.0  11.0

In [64]:
pd.pivot_table(df[df.annAllEffects.str.contains("missense")], values='pos', index=['chromPos','patientName','annGene','meltedAF','numCallers','inCOSMIC'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

consensus                                                            False True   All
chromPos    patientName  annGene meltedAF        numCallers inCOSMIC                 
10:43617398 CPCT02070012 RET     0.0514705882353 2          True              1   1.0
10:89624305 CPCT02110007 PTEN    0.477477477477  4          True              1   1.0
10:89692919 CPCT02070012 PTEN    0.108108108108  3          True              1   1.0
10:89720677 CPCT02060015 PTEN    0.288659793814  4          False             1   1.0
12:25398284 CPCT02010396 KRAS    0.0578512396694 2          True              1   1.0
            CPCT02050050 KRAS    0.0852713178295 2          True              1   1.0
            CPCT02060016 KRAS    0.0469798657718 1          True         1        1.0
            CPCT02110006 KRAS    0.16814159292   4          True              1   1.0
13:32910968 CPCT02030261 BRCA2   0.34693877551   4          False             1   1.0
13:49033926 CPCT02020357 RB1     0.825396825397  4          True              1   1.0
16:68842328 CPCT02050048 CDH1    0.552           4          False             1   1.0
17:37881000 CPCT02060019 ERBB2   0.326086956522  4          True              1   1.0
17:7577120  CPCT02070012 TP53    0.10752688172   2          True              1   1.0
17:7577536  CPCT02110006 TP53    0.23595505618   4          True              1   1.0
17:7577538  CPCT02010390 TP53    0.451612903226  4          True              1   1.0
17:7577539  CPCT02050048 TP53    0.74358974359   4          True              1   1.0
17:7577557  CPCT02010394 TP53    0.0851063829787 1          True         1        1.0
17:7577566  CPCT02020293 TP53    0.629032258065  3          True              1   1.0
17:7577580  CPCT02020334 TP53    0.191919191919  4          True              1   1.0
17:7578268  CPCT02110007 TP53    0.52688172043   4          True              1   1.0
17:7578393  CPCT02140015 TP53    0.253968253968  4          True              1   1.0
17:7578454  CPCT02020348 TP53    0.41935483871   4          True              1   1.0
17:7578523  CPCT02020351 TP53    0.365384615385  4          True              1   1.0
17:7578528  CPCT02020350 TP53    0.465753424658  4          True              1   1.0
1:115256529 CPCT02010393 NRAS    0.856209150327  4          True              1   1.0
1:115256530 CPCT02020358 NRAS    0.48            4          True              1   1.0
2:16082244  CPCT02160001 MYCN    0.316455696203  4          True              1   1.0
2:209113113 CPCT02020358 IDH1    0.25            4          True              1   1.0
3:178916930 CPCT02100027 PIK3CA  0.230215827338  4          True              1   1.0
3:178916947 CPCT02140015 PIK3CA  0.284615384615  4          True              1   1.0
3:178936082 CPCT02030261 PIK3CA  0.244047619048  4          True              1   1.0
3:178936091 CPCT02040035 PIK3CA  0.345323741007  4          True              1   1.0
3:178936095 CPCT02110007 PIK3CA  0.256097560976  4          True              1   1.0
3:178952085 CPCT02030261 PIK3CA  0.438356164384  4          True              1   1.0
3:41266097  CPCT02020360 CTNNB1  0.0519480519481 2          True              1   1.0
3:41266136  CPCT02060014 CTNNB1  0.0549450549451 2          True              1   1.0
4:55141084  CPCT02010325 PDGFRA  0.532110091743  4          False             1   1.0
5:112175180 CPCT02140013 APC     0.186274509804  4          False             1   1.0
6:152419923 CPCT02060022 ESR1    0.420289855072  4          True              1   1.0
6:152419926 CPCT02160002 ESR1    0.237623762376  4          True              1   1.0
7:128846040 CPCT02070012 SMO     0.0733944954128 2          True              1   1.0
7:140453136 CPCT02010397 BRAF    0.514285714286  4          True              1   1.0
            CPCT02020293 BRAF    0.38064516129   4          True              1   1.0
            CPCT02020360 BRAF    0.135338345865  4          True              1   1.0
            CPCT02060015 BRAF    0.420634920

### SINGLE PATIENT ANALYSIS

In [90]:
df[(df.patientName=='CPCT02140013')]#[['patientName','TUMOR_TYPE','pos','chrom','id','variantType','ref','meltedallele','meltedDP','meltedAF','vennSegment','annGene']].head(20)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedindelPos,patientName,TUMOR_PERCENTAGE,TUMOR_TYPE,PIPELINE_VERSION
0,3,10183725,3:10183725,3.051427,.,CGG,varscan-freebayes,2,INDEL,DELETE,PASS,VHL,False,False,VHL,HIGH,frameshift_variant,frameshift_variant|frameshift_variant,True,CG,0.145161,62,-1.0,0/1,G-,-2,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f
1,5,112175180,5:112175180,5.620043,.,G,Intersection,4,SNP,,PASS,APC,False,False,APC,MODERATE,missense_variant,missense_variant|missense_variant|missense_var...,True,A,0.186275,102,-1.0,0/1,,-1,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f
2,10,89720633,10:89720633,10.661975,.,CTTTTTTTTTTTTTTTA,freebayes,1,INDEL,DELETE,.,PTEN,False,False,PTEN,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,CTTTTTTTTTTTTTTA,0.147059,84,-1.0,0/1,T-,-2,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f


### SCRATCH

In [94]:
pd.pivot_table(df, values='pos', index=['numCallers','vennSegment'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

consensus                            False True   All
numCallers vennSegment                               
1          freebayes                     6        6.0
           mutect                        4        4.0
           strelka                       1        1.0
           varscan                       1        1.0
2          strelka-mutect                2   12  14.0
           varscan-freebayes                  2   2.0
3          strelka-varscan-freebayes          9   9.0
           strelka-varscan-mutect             2   2.0
           varscan-freebayes-mutect           1   1.0
4          Intersection                      53  53.0
All                                     14   79  93.0

In [95]:
df[df.numCallers ==1]

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedindelPos,patientName,TUMOR_PERCENTAGE,TUMOR_TYPE,PIPELINE_VERSION
4,10,89692731,10:89692731,10.661769,rs77494260,GT,varscan,1,INDEL,DELETE,PASS,PTEN,True,False,PTEN,LOW,sequence_feature,sequence_feature|sequence_feature|intron_varia...,False,G,0.109244,121,-1.0,0/1,T-,-2,CPCT02070012,30,Prostate cancer,v1.8
0,4,1795557,4:1795557,4.009393,rs10023340,T,freebayes,1,SNP,,.,FGFR3,True,False,FGFR3,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,C,1.000000,17,-1.0,1/1,,-1,CPCT02050050,40,Liver cancer,v1.9
1,10,89720633,10:89720633,10.661975,.,CTTTTTTTTTTTTTTTA,freebayes,1,INDEL,INSERT,.,PTEN,False,False,PTEN,HIGH,splice_acceptor_variant&intron_variant,splice_acceptor_variant&intron_variant|splice_...,False,CTTTTTTTTTTTTTTTTA,0.148649,91,-1.0,0/1,T+,-2,CPCT02160002,80,Breast cancer,v1.8-50-g2ddb75f
2,10,89720633,10:89720633,10.661975,.,CTTTTTTTTTTTTTTTA,freebayes,1,INDEL,DELETE,.,PTEN,False,False,PTEN,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,CTTTTTTTTTTTTTTA,0.147059,84,-1.0,0/1,T-,-2,CPCT02140013,30,Unknown,v1.8-50-g2ddb75f
1,4,1795557,4:1795557,4.009393,rs10023340,T,freebayes,1,SNP,,.,FGFR3,True,False,FGFR3,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,C,1.000000,30,-1.0,1/1,,-1,CPCT02060014,40,Unknown,v1.8-50-g2ddb75f
2,17,7577552,17:7577552,17.093325,.,CAT,freebayes,1,INDEL,DELETE,.,TP53,False,False,TP53,HIGH,frameshift_variant,frameshift_variant|frameshift_variant|frameshi...,False,CT,0.208000,126,-1.0,0/1,A-,-2,CPCT02060014,40,Unknown,v1.8-50-g2ddb75f
5,17,41245237,17:41245237,17.507976,rs16940;COSM3755565,A,strelka,1,SNP,,PASS,BRCA1,True,True,BRCA1,LOW,synonymous_variant,synonymous_variant|synonymous_variant|synonymo...,False,G,0.039216,103,-1.0,0/1,,-1,CPCT02060015,50,Unknown,v1.9
0,17,7577557,17:7577557,17.093325,COSM44935,A,mutect,1,SNP,,PASS,TP53,False,True,TP53,MODERATE,missense_variant,missense_variant|missense_variant|missense_var...,False,T,0.085106,94,-1.0,0/1,,-1,CPCT02010394,30,Sarcoma,v1.8-50-g2ddb75f
0,4,1795557,4:1795557,4.009393,rs10023340,T,freebayes,1,SNP,,.,FGFR3,True,False,FGFR3,LOW,splice_region_variant&intron_variant,splice_region_variant&intron_variant|splice_re...,False,C,1.000000,5,-1.0,1/1,,-1,CPCT02020293,40,Colon and Rectal cancer,v1.9
0,12,25398284,12:25398284,12.189749,rs121913529;COSM521,C,mutect,1,SNP,,PASS,KRAS,True,True,KRAS,MODERATE,missense_variant,missense_variant|missense_variant|missense_var...,False,T,0.046980,150,-1.0,0/1,,-1,CPCT02060016,30,Unknown,v1.9


In [97]:
df[df.patientName=='CPCT02010393']

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedindelPos,patientName,TUMOR_PERCENTAGE,TUMOR_TYPE,PIPELINE_VERSION
0,1,115256529,1:115256529,1.462412,rs11554290;COSM584,T,Intersection,4,SNP,,PASS,NRAS,True,True,NRAS,MODERATE,missense_variant,missense_variant|sequence_feature,True,C,0.856209,156,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9
1,3,178938876,3:178938876,3.903629,.,C,Intersection,4,SNP,,PASS,PIK3CA,False,False,PIK3CA,LOW,synonymous_variant,synonymous_variant|sequence_feature|non_coding...,True,T,0.524476,143,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9
2,6,152420086,6:152420086,6.890746,.,C,Intersection,4,SNP,,PASS,ESR1,False,False,ESR1,LOW,synonymous_variant,synonymous_variant|synonymous_variant|synonymo...,True,T,0.672414,175,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9
3,7,55223550,7:55223550,7.347015,.,C,Intersection,4,SNP,,PASS,EGFR,False,False,EGFR,MODERATE,missense_variant,missense_variant|missense_variant|missense_var...,True,T,0.136986,147,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9
4,9,21971154,9:21971154,9.155588,.,CGC,strelka-varscan-freebayes,3,INDEL,DELETE,PASS,CDKN2A,False,False,CDKN2A,HIGH,frameshift_variant,frameshift_variant|frameshift_variant|frameshi...,True,CC,0.772727,68,-1.0,0/1,G-,-2,CPCT02010393,40,Melanoma,v1.9
5,13,28626717,13:28626717,13.248561,.,G,Intersection,4,SNP,,PASS,FLT3,False,False,FLT3,LOW,synonymous_variant,synonymous_variant|synonymous_variant|synonymo...,True,A,0.111940,134,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9
6,17,7578212,17:7578212,17.093333,COSM10654,G,Intersection,4,SNP,,PASS,TP53,False,True,TP53,HIGH,stop_gained,stop_gained|stop_gained|stop_gained|stop_gaine...,True,A,0.826446,121,-1.0,0/1,,-1,CPCT02010393,40,Melanoma,v1.9


In [129]:
temp = df[df.bedRegion=='Default'][['chrom','pos','bedRegion','annGene','annWorstEffect','chromPos','id']].groupby(['chrom','pos','bedRegion','annGene','annWorstEffect','id']).agg('count').sort_values('chromPos',ascending=False)
temp = temp[temp.chromPos>1]

In [143]:
temp = pd.pivot_table(df[(df.bedRegion=='Default')&((df.annWorstImpact=='HIGH')|(df.annWorstImpact=='MODERATE'))], values='chromPos', index=['chrom','pos','annGene','annWorstEffect','id'], columns=['bedRegion'], aggfunc='count',margins=True).fillna("")
len(temp)

8705

In [127]:
#temp.reset_index
#groupby(['bedRegion']).agg('count')

In [145]:
temp.sort_values('All',ascending=False)

,,,,bedRegion,Default,All
chrom,pos,annGene,annWorstEffect,id,,
All,,,,,9346.0,9346.0
X,106846459,FRMPD3,stop_gained&inframe_insertion,.,16.0,16.0
16,57562804,CCDC102A,missense_variant,rs12935069;COSM3754954,11.0,11.0
14,20528448,OR4L1,frameshift_variant,rs112192573;COSM128642,10.0,10.0
16,89952385,TCF25,frameshift_variant,.,8.0,8.0
19,50832152,KCNC3,missense_variant,COSM4132308,8.0,8.0
17,45216247,CDC27,missense_variant,rs201607934,7.0,7.0
2,6789852,hsa-mir-7515,splice_acceptor_variant&splice_region_variant&intron_variant&non_coding_exon_variant,rs111797680,7.0,7.0
17,28036035,SSH2,sequence_feature,rs3115091,6.0,6.0
